THIS NOTEBOOK USES GEMINI AS THE PREFERED LLM 

In [13]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env file
load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")

# Now initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,
    temperature=0.0
)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [15]:
template_1 = """You are a video game expert. Recommend 5 {genre} games similar to {type}.

Requirements:
- Only list game titles (one per line)
- No descriptions, ratings, or extra text
- Focus on critically acclaimed games
- Include both classic and modern titles

Format:
1. Game Title
2. Game Title
..."""

template_2 = """Below is a list of video games:

{games}

For each game above, provide:
- A 1-2 sentence description
- Key gameplay features or what makes it unique

Format each entry as:
**Game Title**: Description here.

Important: Only describe the games listed. Do not recommend additional titles."""



In [16]:
from langchain.prompts import PromptTemplate

# Template for game recommendation
prompt_temp_genre_1 = PromptTemplate(
    input_variables=["genre"],
    template=template_1,
    partial_variables= {"type" : ""} # This is a optional variable 
)

# Template for description generation
prompt_temp_genre_2 = PromptTemplate(
    input_variables=["games"],
    template=template_2,
)


Simple Chain

In [17]:
# SIMPLE CHAIN

chain_1 = prompt_temp_genre_1 | llm | StrOutputParser()
chain_2 = prompt_temp_genre_2 | llm | StrOutputParser()

result = chain_1.invoke({"genre": "rpg", "type": "souls" })
print(result)

Bloodborne
Nioh 2
Lies of P
Dragon's Dogma: Dark Arisen
Monster Hunter: World


Simple Sequential chain

In [18]:
seq_chain = chain_1 | (lambda games: {"games": games}) | chain_2

result_seq = seq_chain.invoke({"genre": "rpg"})
result_seq_2 = seq_chain.invoke({"genre": "rpg", "type": "souls"})
print(result_seq)

**The Witcher 3: Wild Hunt**: An action role-playing game set in a vast, open fantasy world, where players embody Geralt of Rivia, a monster hunter for hire, searching for his adopted daughter Ciri. It's renowned for its mature storytelling, complex moral choices, and rich lore, offering a truly immersive experience.
*Key features include open-world exploration, a deep narrative with branching quests, real-time combat combining swordplay, magic (Signs), and alchemy, and a vast world filled with monster hunting contracts and side activities.*

**Persona 5 Royal**: A stylish Japanese role-playing game where players control a high school student who, along with a group of fellow outcasts, forms the "Phantom Thieves of Hearts" to change corrupt adults' hearts. It blends social simulation with dungeon crawling, all presented with a unique aesthetic.
*Key features include a turn-based combat system with "1 More" and "All-Out Attack" mechanics, Persona collection and fusion, a day-by-day cale

Sequential chain or combined chain ( output of 1st chain is input of the 2nd)

In [ ]:
game_chain = prompt_temp_genre_1 | llm | StrOutputParser()
des_chain = prompt_temp_genre_2 | llm | StrOutputParser()

combined_chain = (
    game_chain  # Step 1

    | (lambda games: {  # Step 2 (depends on Step 1)
        "games": games,

        "descriptions": des_chain.invoke({"games": games})  # Step 3 (depends on Step 1)
    })
)

result = combined_chain.invoke({"genre": "rpg", "type": "souls"})

print("Games:\n", result["games"])
print("\nDescriptions:\n", result["descriptions"])

Games:
 Bloodborne
Nioh 2
Lies of P
Dragon's Dogma: Dark Arisen
Monster Hunter: World

Descriptions:
 **Bloodborne**: A gothic horror action RPG from FromSoftware, known for its fast-paced, aggressive combat and Lovecraftian themes. Its unique features include trick weapons that transform mid-combat, a focus on regaining health by attacking, and a challenging parry system using firearms.

**Nioh 2**: An intense action RPG set in a fictionalized Sengoku-era Japan, where players battle both human and Yokai enemies. Key gameplay elements include a versatile stance system for combat, the ability to utilize Yokai abilities and transformations, and a deep loot system with extensive character customization.

**Lies of P**: A soulslike action RPG that reimagines the classic tale of Pinocchio in a dark, Belle Époque-inspired world. It features a unique weapon assembly system, versatile Legion Arms with various abilities, and a "Lie" system where player choices impact the narrative and ending.

